In [4]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

import re
import pandas as pd
from pprint import pprint

from gensim.models import Word2Vec
from gensim.models import KeyedVectors
from gensim.test.utils import datapath
from scipy.spatial.distance import cosine

import gsd.gene_sets
from functools import reduce 

In [3]:
df = pd.read_table("__data/gwas/gwas_catalog_v1.0-associations_e93_r2018-12-21.tsv")
print(df.columns)
df.loc[df['MAPPED_GENE'] == "SNCA","DISEASE/TRAIT"]

Index(['DATE ADDED TO CATALOG', 'PUBMEDID', 'FIRST AUTHOR', 'DATE', 'JOURNAL',
       'LINK', 'STUDY', 'DISEASE/TRAIT', 'INITIAL SAMPLE SIZE',
       'REPLICATION SAMPLE SIZE', 'REGION', 'CHR_ID', 'CHR_POS',
       'REPORTED GENE(S)', 'MAPPED_GENE', 'UPSTREAM_GENE_ID',
       'DOWNSTREAM_GENE_ID', 'SNP_GENE_IDS', 'UPSTREAM_GENE_DISTANCE',
       'DOWNSTREAM_GENE_DISTANCE', 'STRONGEST SNP-RISK ALLELE', 'SNPS',
       'MERGED', 'SNP_ID_CURRENT', 'CONTEXT', 'INTERGENIC',
       'RISK ALLELE FREQUENCY', 'P-VALUE', 'PVALUE_MLOG', 'P-VALUE (TEXT)',
       'OR or BETA', '95% CI (TEXT)', 'PLATFORM [SNPS PASSING QC]', 'CNV'],
      dtype='object')


/opt/local/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (23,27) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,DATE ADDED TO CATALOG,PUBMEDID,FIRST AUTHOR,DATE,JOURNAL,LINK,STUDY,DISEASE/TRAIT,INITIAL SAMPLE SIZE,REPLICATION SAMPLE SIZE,...,CONTEXT,INTERGENIC,RISK ALLELE FREQUENCY,P-VALUE,PVALUE_MLOG,P-VALUE (TEXT),OR or BETA,95% CI (TEXT),PLATFORM [SNPS PASSING QC],CNV
31164,2009-12-10,19915575,Simon-Sanchez J,2009-11-15,Nat Genet,www.ncbi.nlm.nih.gov/pubmed/19915575,Genome-wide association study reveals genetic ...,Parkinson's disease,"1,713 European ancestry cases, 3,978 European ...","3,361 European ancestry cases, 4,573 European ...",...,intron_variant,0.0,0.51,2e-16,15.698970,NaN,1.230000,[NR],Illumina [463185],N
31351,2010-02-05,20070850,Edwards TL,2010-01-13,Ann Hum Genet,www.ncbi.nlm.nih.gov/pubmed/20070850,Genome-wide association study confirms SNPs in...,Parkinson's disease,"1,752 European ancestry cases, 1,745 European ...",NaN,...,intron_variant,0.0,NR,7e-08,7.154902,NaN,1.290000,[1.18-1.43],Illumina [495715] (imputed),N
37676,2017-03-17,27089181,Okbay A,2016-04-18,Nat Genet,www.ncbi.nlm.nih.gov/pubmed/27089181,Genetic variants associated with subjective we...,Neuroticism,"170,911 European ancestry individuals",NaN,...,intron_variant,0.0,0.207,2E-6,5.698970,NaN,0.020366,[0.012-0.029] unit decrease,Affymetrix [up to 6544775] (imputed),N
63960,2018-02-09,29263008,Guerreiro R,2018-01-01,Lancet Neurol,www.ncbi.nlm.nih.gov/pubmed/29263008,Investigating the genetic architecture of deme...,Dementia with Lewy bodies,"1,216 European ancestry cases, 3,791 European ...","527 European ancestry cases, 663 European ance...",...,intron_variant,0.0,0.52,9e-13,12.045757,NaN,1.369863,[1.27-1.49],Illumina [8397716] (imputed),N
97278,2018-11-16,30072576,Emilsson V,2018-08-02,Science,www.ncbi.nlm.nih.gov/pubmed/30072576,Co-regulatory networks of human serum proteins...,Blood protein levels,"3,200 European ancestry individuals",NaN,...,5_prime_UTR_variant,0.0,0.775,8E-30,29.096910,(SNCA),0.330000,[0.27-0.39] unit decrease,Illumina [> 1000000] (imputed),N
97279,2018-11-16,30072576,Emilsson V,2018-08-02,Science,www.ncbi.nlm.nih.gov/pubmed/30072576,Co-regulatory networks of human serum proteins...,Blood protein levels,"3,200 European ancestry individuals",NaN,...,intron_variant,0.0,0.791,1E-9,9.000000,(SNCA),0.181000,[0.12-0.24] unit decrease,Illumina [> 1000000] (imputed),N
98620,2018-10-31,29942085,Nagel M,2018-06-25,Nat Genet,www.ncbi.nlm.nih.gov/pubmed/29942085,Meta-analysis of genome-wide association studi...,Neuroticism,"449,484 European ancestry individuals",NaN,...,intron_variant,0.0,NR,8e-10,9.096910,NaN,6.142000,z score increase,Affymetrix [10847151] (imputed),N
98869,2018-10-31,29942085,Nagel M,2018-06-25,Nat Genet,www.ncbi.nlm.nih.gov/pubmed/29942085,Meta-analysis of genome-wide association studi...,Worry,"348,219 European ancestry individuals",NaN,...,3_prime_UTR_variant,0.0,NR,3e-10,9.522879,NaN,0.015080,[0.01-0.02] unit decrease,Affymetrix [10847151] (imputed),N
99011,2018-09-20,30014611,Weiss RB,2018-07-16,Ann Neurol,www.ncbi.nlm.nih.gov/pubmed/30014611,Long-range genomic regulators of THBS1 and LTB...,Age at loss of ambulation in Duchenne muscular...,"243 European ancestry cases, 10 cases",NaN,...,intron_variant,0.0,NR,6e-06,5.221849,NaN,NaN,NaN,Illumina [1735619],N
